In [ ]:
import json
import time
import os
import requests

import fml_manager
from fml_manager import *

manager = fml_manager.FMLManager()

## For more details about the FMLManager, please refer to this [document](https://kubefate.readthedocs.io/README.html)

In [ ]:
response = manager.load_data(url='./data/breast_b.csv', namespace='fate_flow_test_breast', table_name='breast_b', work_mode=1, head=1, partition=10)
output = json.loads(response.content)
print(output)
guest_job_id = output['jobId']
guest_query_condition = QueryCondition(job_id=guest_job_id)

manager.query_job_status(guest_query_condition, max_tries=20)

In [ ]:
# Pipeline
data_io = ComponentBuilder(name='dataio_0',
                           module='DataIO')\
                           .add_input_data('args.train_data')\
                           .add_output_data('train')\
                           .add_output_model('dataio').build()
        
hetero_feature_binning = ComponentBuilder(name='hetero_feature_binning_0',
                                          module='HeteroFeatureBinning')\
                                          .add_input_data('dataio_0.train')\
                                          .add_output_data('train')\
                                          .add_output_model('hetero_feature_binning').build()

hetero_feature_selection = ComponentBuilder(name='hetero_feature_selection_0',
                                            module='HeteroFeatureSelection')\
                                            .add_input_data('hetero_feature_binning_0.train')\
                                            .add_input_isometric_model('hetero_feature_binning_0.hetero_feature_binning')\
                                            .add_output_data('train')\
                                            .add_output_model('selected').build()

hetero_lr = ComponentBuilder(name='hetero_lr_0',
                             module='HeteroLR')\
                             .add_input_train_data('hetero_feature_selection_0.train')\
                             .add_output_data('train')\
                             .add_output_model('hetero_lr').build()

evaluation = ComponentBuilder(name='evaluation_0',
                              module='Evaluation',
                              need_deploy=False)\
                              .add_input_data('hetero_lr_0.train')\
                              .add_output_data('evaluate').build()
pipeline = Pipeline(
    data_io, 
    hetero_feature_selection,  
    hetero_feature_binning, 
    hetero_lr, 
    evaluation)

# Configuration

initiator = Initiator(role='guest', party_id=10000)

job_parameters = JobParameters(work_mode=1)

role = RoleBuilder()\
    .add_guest(party_id=10000)\
    .add_host(party_id=9999)\
    .add_arbiter(party_id=9999).build()

guest_data_io_config = {
    'with_label': [True],
    'label_name': ['y'],
    'label_type': ['int'],
    'output_format': ['dense']
}

host_data_io_config = {
    'with_label': [False],
    'output_format': ['dense']
}

role_parameters = RoleParametersBuilder()\
    .add_guest_train_data(namespace='fate_flow_test_breast', name='breast_b')\
    .add_guest_module_config(module='dataio_0', config=guest_data_io_config)\
    .add_host_train_data(namespace='fate_flow_test_breast', name='breast_a')\
    .add_host_module_config(module='dataio_0', config=host_data_io_config).build()

hetero_lr_params = {
    'penalty': 'L2',
    'optimizer': 'rmsprop',
    'eps': 1e-5,
    'alpha': 0.01,
    'max_iter': 3,
    'converge_func': 'diff',
    'batch_size': 320,
    'learning_rate': 0.15,
    'init_param': {
        'init_method': 'random_uniform'
    }
}

algorithm_parameters = AlgorithmParametersBuilder()\
    .add_module_config(module='hetero_lr_0', config=hetero_lr_params).build()

config = Config(initiator, job_parameters, role, role_parameters, algorithm_parameters)

In [ ]:
response = manager.submit_job(pipeline.to_dict(),config.to_dict())
manager.prettify(response, verbose=True)
stdout = json.loads(response.content)
job_id = stdout['jobId']
query_condition = QueryCondition(job_id)
model_id, model_version = '', ''
manager.query_job_status(query_condition, max_tries=20)

manager.prettify(response, verbose=True)
output = json.loads(response.content)
model_id, model_version = output['data']['model_info']['model_id'], output['data']['model_info']['model_version']

## Click [here](/fateboard-10000/) to view jobs in FATE Board

In [ ]:
# Pipline
data_io = ComponentBuilder(name='dataio_0',
                           module='DataIO')\
                           .add_input_data('args.train_data')\
                           .add_output_data('train')\
                           .add_output_model('dataio').build()

hetero_feature_binning = ComponentBuilder(name='hetero_feature_binning_0',
                                          module='HeteroFeatureBinning')\
                                          .add_input_data('dataio_0.train')\
                                          .add_output_data('train')\
                                          .add_output_model('hetero_feature_binning').build()        

hetero_feature_selection = ComponentBuilder(name='hetero_feature_selection_0',
                                            module='HeteroFeatureSelection')\
                                            .add_input_data('hetero_feature_binning_0.train')\
                                            .add_input_isometric_model('hetero_feature_binning_0.hetero_feature_binning')\
                                            .add_output_data('eval')\
                                            .add_output_model('selected').build()

evaluation = ComponentBuilder(name='evaluation_0',
                              module='Evaluation',
                              need_deploy=False)\
                              .add_input_data('hetero_feature_selection_0.eval')\
                              .add_output_data('evaluate').build()

pipeline = Pipeline(
    data_io, 
    hetero_feature_selection,  
    hetero_feature_binning, 
    evaluation)

# Configuration
initiator = Initiator(role='guest', party_id=10000)

job_parameters = JobParameters(work_mode=1,
                               job_type='predict',
                               model_id='arbiter-9999#guest-10000#host-9999#model',
                               model_version='2020060802475836992436')

role = RoleBuilder()\
    .add_guest(party_id=10000)\
    .add_host(party_id=9999)\
    .add_arbiter(party_id=9999).build()

role_parameters = RoleParametersBuilder()\
    .add_guest_eval_data(namespace='fate_flow_test_breast', name='breast_b')\
    .add_host_eval_data(namespace='fate_flow_test_breast', name='breast_a').build()

config = Config(initiator, job_parameters, role, role_parameters)

In [ ]:
response = manager.submit_job(pipeline.to_dict(), config.to_dict())
manager.prettify(response, verbose=True)
stdout = json.loads(response.content)
job_id = stdout['jobId']
query_condition = QueryCondition(job_id)

In [ ]:
print(query_condition)

In [ ]:
model_id, model_version = '', ''
manager.query_job_status(query_condition, max_tries=20)

manager.prettify(response, verbose=True)
output = json.loads(response.content)
model_id, model_version = output['data']['model_info']['model_id'], output['data']['model_info']['model_version']

In [ ]:
# data is a pandas.DataFrame
data = manager.track_component_output_data(job_id='2020060806580950942753', role='guest', party_id='10000', component_name='hetero_lr_0')

print(data.head(4))